In [1]:
import pandas as pd
import numpy as np

DATA_LOC = "Data/completeOdds.csv"

YEARS = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

complete_odds = pd.read_csv(DATA_LOC)
complete_odds.drop(columns=complete_odds.columns[0], inplace=True)

### Visualize and Clean 

In [2]:
display(complete_odds.head(5))

,Date,AwayTeam,HomeTeam,AwayFirst,AwaySecond,AwayThird,AwayFourth,AwayFinal,AwayML,HomeFirst,HomeSecond,HomeThird,HomeFourth,HomeFinal,HomeML,Spread,Favorite,TotalPoints
0,1030.0,Washington,Cleveland,24.0,15.0,23.0,22.0,84.0,210.0,27.0,23.0,24.0,20.0,94.0,-250.0,6.0,Home,190.5
1,1030.0,Boston,Miami,25.0,29.0,22.0,31.0,107.0,240.0,31.0,31.0,31.0,27.0,120.0,-280.0,6.5,Home,187.5
2,1030.0,Dallas,LALakers,25.0,23.0,26.0,25.0,99.0,400.0,29.0,17.0,20.0,25.0,91.0,-500.0,8.5,Home,186.5
3,1031.0,Indiana,Toronto,27.0,21.0,15.0,27.0,90.0,-120.0,25.0,23.0,23.0,17.0,88.0,100.0,1.0,Away,190.0
4,1031.0,Denver,Philadelphia,22.0,16.0,14.0,23.0,75.0,-130.0,25.0,21.0,19.0,19.0,84.0,110.0,2.0,Away,197.5


In [3]:
def get_month(date):
    if len(date) == 4:
        return int(date[:2])
    else:
        return int(date[0])

def split_month_day(df):
    # eliminate the decimals
    df["Date"] = df["Date"].astype(str)
    df["Date"] = df["Date"].str[:-2]
    
    # the last two digits must be the day
    df["day"] = df["Date"].str[-2:].astype(int)
    
    # the first one or two must be the month
    df["month"] = df["Date"].apply(lambda row: get_month(row))
    return df

def add_years(df):
    # find instances where it turns into a new year
    new_years = df["month"] == 1
    year_idx = 0
    years = np.zeros((len(df), 1))
    for index, row in df.iterrows():
        if row["month"] == 1 and df.iloc[index-1, :]["month"] == 12:
            year_idx += 1
        years[index] = YEARS[year_idx]
    df["year"] = years
    
    return df

def create_date_time(df):
    df["Date"] = pd.to_datetime(df[["month", "day", "year"]])
    return df

In [4]:
def pre_process(df):
    df = split_month_day(df)
    df = add_years(df)
    df = create_date_time(df)
    return df

odds_cleaned = pre_process(complete_odds.copy(deep=True))

In [5]:
display(odds_cleaned.head(5))

,Date,AwayTeam,HomeTeam,AwayFirst,AwaySecond,AwayThird,AwayFourth,AwayFinal,AwayML,HomeFirst,...,HomeThird,HomeFourth,HomeFinal,HomeML,Spread,Favorite,TotalPoints,day,month,year
0,2012-10-30,Washington,Cleveland,24.0,15.0,23.0,22.0,84.0,210.0,27.0,...,24.0,20.0,94.0,-250.0,6.0,Home,190.5,30,10,2012.0
1,2012-10-30,Boston,Miami,25.0,29.0,22.0,31.0,107.0,240.0,31.0,...,31.0,27.0,120.0,-280.0,6.5,Home,187.5,30,10,2012.0
2,2012-10-30,Dallas,LALakers,25.0,23.0,26.0,25.0,99.0,400.0,29.0,...,20.0,25.0,91.0,-500.0,8.5,Home,186.5,30,10,2012.0
3,2012-10-31,Indiana,Toronto,27.0,21.0,15.0,27.0,90.0,-120.0,25.0,...,23.0,17.0,88.0,100.0,1.0,Away,190.0,31,10,2012.0
4,2012-10-31,Denver,Philadelphia,22.0,16.0,14.0,23.0,75.0,-130.0,25.0,...,19.0,19.0,84.0,110.0,2.0,Away,197.5,31,10,2012.0


### Export to a CSV for later

In [6]:
odds_cleaned.to_csv('Data/cleaned_odds.csv')